In [2]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
import tracks_interactions.graph.family_graph as fg
#from tracks_interactions.widget.event_handler import EventHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
os.environ['NAPARI_PERFMON'] = '0'

In [4]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)
labels.shape

(241, 8396, 8401)

In [4]:
#labels = np.array(labels).astype('int32')

In [10]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')

In [6]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables - Copy.db'
engine = create_engine(f'sqlite:///{new_db_path}')
session = sessionmaker(bind=engine)()

In [11]:
# from napari import Viewer

# def test_func(viewer: Viewer):
#     print('just testing')

# labels_layer.events.labels_update.connect(test_func)

In [11]:
from tracks_interactions.graph.family_graph import FamilyGraph
plot_widget = FamilyGraph(viewer,session)
viewer.window.add_dock_widget(plot_widget,area='bottom')

In [32]:
from tracks_interactions.widget.tracks_widget import TracksWidget
tracks_widget = TracksWidget(viewer,session)
viewer.window.add_dock_widget(tracks_widget,area='right')

In [33]:
def change_status(layer,event):
    print(event)

labels_layer.events.selected_label.connect(change_status)

RuntimeError: Binning function cannot have more than one positional argument

In [9]:
from magicgui import magicgui
from qtpy.QtGui import QIcon
from napari import Viewer
from tracks_interactions.widget.track_operations import cut_track_function, merge_track_function, connect_track_function
from functools import partial

In [29]:
######################################################################
# add cut tracks button
cut_track_function_partial = partial(cut_track_function, session=session)

cut_track_btn = magicgui(cut_track_function_partial)
path_to_some_icon = r'../tracks_interactions/icons/icons8-scissors-50.png'

icon = QIcon(path_to_some_icon)
cut_track_btn.call_button.native.setIcon(icon)
cut_track_btn.call_button.native.setText(None)

viewer.window.add_dock_widget(cut_track_btn,area='right',name='cut_track')
cut_track_btn.call_button.native.resize(40,40)

In [33]:
# add merging
@magicgui(
        t1={"widget_type": "SpinBox", "max": 1000000},
        call_button = 'Merge'
        )  

def merge_track_function_partial(viewer: Viewer, t1: int):
    return merge_track_function( viewer, session=session, t1=t1)

viewer.window.add_dock_widget(merge_track_function_partial,area='right',name='t1')

In [34]:
# add connecting
@magicgui(
        t1={"widget_type": "SpinBox", "max": 1000000},
        call_button = 'Connect'
        )  

def connect_track_function_partial(viewer: Viewer, t1: int):
    return connect_track_function( viewer, session=session, t1=t1)

viewer.window.add_dock_widget(connect_track_function_partial,area='right',name='t1')

In [27]:
from tracks_interactions.graph.family_graph import update_lineage_display
from tracks_interactions.db.db_functions import (
    cut_trackDB,
    get_descendants,
    integrate_trackDB,
    modify_track_cellsDB,
    newTrack_number,
)

41322
918


In [97]:
ch = 75087
t1 = session.query(TrackDB).filter_by(track_id=ch).first()
t1
    

Track 75087 from 170 to 179

In [22]:
newTrack_number(session)

75065

In [20]:
update_lineage_display(viewer, plot_widget, session)

IndexError: index 0 is out of bounds for axis 0 with size 0